# File orchestration

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Files for testing

In [ ]:
%%luminesce

-- Description:
-- In this example we show how you can manage file 
-- orchestration using Luminesce. If a response/table
-- has errors, we move the file to an "error" directory.
-- If the response/table has no errors, we move the file to
-- a "processed" directory



@@successFileName1 = select 'instruments_success_001';
@@successFileName2 = select 'instruments_success_002';
@@successFileName3 = select 'instruments_success_003';
@@errorFileName1 = select 'instruments_error_001';
@@newDirectory = select 'luminesce-examples/orchestration/new';

/*

==============================================
        1. Create files for testing
==============================================

*/

@successFile1 = select 'BondA' as 'InstrumentId',
0 as WriteErrorCode, @@successFileName1  as FileName;

@successFile2 = select 'BondB' as 'InstrumentId',
0 as WriteErrorCode, @@successFileName2 as FileName;

@successFile3 = select 'BondC' as 'InstrumentId',
0 as WriteErrorCode, @@successFileName3 as FileName;

@errorFile1 = select 'BondD' as 'InstrumentId',
105 as WriteErrorCode, @@errorFileName1 as FileName
union
values
('BondE', 0, 'instruments_error_001.csv');

/*

==============================================
        2. Save new file to Drive
==============================================

*/


@saveFilesToDrive = use Drive.SaveAs with @successFile1, @successFile2, @successFile3, @errorFile1, @@newDirectory
--path=/{@@newDirectory}
--ignoreOnZeroRows=true
--fileNames
instruments_success_001
instruments_success_002
instruments_success_003
instruments_error_001
enduse;

select * from @saveFilesToDrive;


#### Step 2: Create error file

In [ ]:
%%luminesce

-- Description:
-- In this example we show how you can manage file 
-- orchestration using Luminesce. If a response/table
-- has errors, we move the file to an "error" directory.
-- If the response/table has no errors, we move the file to
-- a "processed" directory


@@fileName = select 'instruments_error.csv';
@@fileNameNoExtension = select substr(@@fileName, 1, length(@@fileName)-4);
@@newDirectory = select 'luminesce-examples/orchestration/new';

/*

==============================================
        1. Create success file for testing
==============================================

NOTE: 
    Run either 1A or 1B
    1A uses a file with no errors
    1B uses a file with errors

*/

@instrumentsResponse = select 
'BondA' as 'InstrumentId',
0 as WriteErrorCode -- success
union all 
values
('BondB', 102),
('BondC', 102),
('BondD', 103);


/*

==============================================
        2. Save new file to Drive
==============================================

*/


@saveFilesToDrive = use Drive.SaveAs with @instrumentsResponse, @@newDirectory, @@fileNameNoExtension 
--path=/{@@newDirectory}
--ignoreOnZeroRows=true
--fileNames={@@fileNameNoExtension}
enduse;

select * from @saveFilesToDrive;


#### Step 3: Create logger view

In [ ]:
%%luminesce

-- Description:
-- Create a view to save logs

@logs = select 
'Equity' as [Instrument],
0 as [WriteErrorCode];

@create_logger_view = use Sys.Admin.SetupView with @logs
--provider=ETL_Manager.Save_Logs_To_Drive
--description="This is a tool for saving logs"
--parameters
LogsLocation,Text,"luminesce-examples/orchestration/logs",true
LogFileName,Text,"instrument_log_load",true
Logs,table,@logs,true
----

--@logs = select #PARAMETERVALUE(Logs);
@@logsLocation = select #PARAMETERVALUE(LogsLocation);
@@logFileName = select #PARAMETERVALUE(LogFileName) || strftime('%Y%m%d_%H%M%S', datetime());

@errorsFromLoad = select * from 
#PARAMETERVALUE(Logs) where WriteErrorCode<>0;

@@errorCount = select count(*) from @errorsFromLoad;

@saveFilesToDrive = use Drive.SaveAs with @errorsFromLoad, @@logsLocation, @@logFileName
--path=/{@@logsLocation}
--ignoreOnZeroRows=true
--fileNames={@@logFileName}
enduse;

@@throwErrorOnError = select iif(@@errorCount > 0,
print('The load has partially failed with errors'), "File loaded successfully"); --change "print" to "throw"

select * from #PARAMETERVALUE(Logs);

enduse;

select * from @create_logger_view;


#### Step 4: Create file orchestration view

In [ ]:
%%luminesce

-- Description:
-- In this example we show how you can manage file 
-- orchestration using Luminesce. If a response/table
-- has errors, we move the file to an "error" directory.
-- If the response/table has no errors, we move the file to
-- a "processed" directory


@load_results = 
select 
'Equity' as [Instrument],
0 as [WriteErrorCode],
"instruments_success.csv" as [FileName]
union 
values
('Equity', 103, "instruments_error.csv");


@create_file_orchestration_view = use Sys.Admin.SetupView with @load_results
--provider=ETL_Manager.File_Orchestration
--description="This is a tool for moving files based off load results"
--parameters
NewDirectory,Text,"luminesce-examples/orchestration/new",true
ErrorDirectory,Text,"luminesce-examples/orchestration/error",true
ProcessedDirectory,Text,"luminesce-examples/orchestration/processed",true
LoadResults,table,@load_results,true
----

@@newDirectory = select #PARAMETERVALUE(NewDirectory);
@@processedDirectory = select #PARAMETERVALUE(ProcessedDirectory);
@@errorDirectory = select #PARAMETERVALUE(ErrorDirectory);
@loadResults = select * from #PARAMETERVALUE(LoadResults);

@filesWithErrors = 
select distinct FileName, 
@@errorDirectory as TargetDirectory
from @loadResults where WriteErrorCode<>0;

@fileWithNoErrors = select 
distinct FileName,
@@processedDirectory as TargetDirectory
from @loadResults where FileName not in (select FileName from @filesWithErrors);

@filesWithTarget = 
select * from @filesWithErrors 
union all
select * from @fileWithNoErrors;

@filePathsToMove = 
select 
@@newDirectory || '/' || FileName as FullPath,
TargetDirectory || '/' || FileName as NewFullPath,
'MoveRenameMayOverwrite' as Operation
from @filesWithTarget;

@upload_files_to_drive = select *
from Drive.file.Operation
where UpdatesToPerform = @filePathsToMove;

enduse;

select * from @create_file_orchestration_view;


#### Step 5: Run file load

In [ ]:
%%luminesce

@@logsDirectory = select 'luminesce-examples/orchestration/logs';
@@newDirectory = select 'luminesce-examples/orchestration/new';
@@errorDirectory = select 'luminesce-examples/orchestration/error';
@@processedDirectory = select 'luminesce-examples/orchestration/processed';

@instrumentsResponse = select 
'BondA' as 'InstrumentId',
0 as WriteErrorCode, 'instruments_success_001.csv' as FileName
union
values
('BondB', 0, 'instruments_success_002.csv'),
('BondC', 0, 'instruments_success_003.csv'),
('BondD', 105, 'instruments_error_001.csv'),
('BondE', 0, 'instruments_error_001.csv');


/*

==============================================
        1. Run file orchestration
==============================================

*/

@runFileOrchestration = select * from 
ETL_Manager.File_Orchestration
where  NewDirectory = @@newDirectory
and ErrorDirectory = @@errorDirectory
and ProcessedDirectory = @@processedDirectory
and LoadResults = @instrumentsResponse;

/*

==============================================
        2. Save logs
==============================================

*/

@saveLogs = select * from ETL_Manager.Save_Logs_To_Drive
where Logs = @instrumentsResponse
and LogFileName = 'instruments_load_logs_'
and LogsLocation = @@logsDirectory;

select * from @runFileOrchestration;

